# Retrieving scientific articles for database

## Libraries

In [8]:
from langchain_community.document_loaders import PyPDFLoader, pdf
import os
import re
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import json
from langchain_google_genai import GoogleGenerativeAI
import pickle as pkl
import requests
import subprocess
import pandas as pd
from bs4 import BeautifulSoup
import tqdm
import time
import helper_functions as hf

## Config

In [3]:
os.getcwd()

'/data/local/aparraga/Bioinformatician/RAG'

In [4]:
with open('api_google.txt') as f:
    
    api_key = json.load(f)

## Functions

## Timing

In [6]:
time_management = {}

## LLM Model

In [ ]:
llm = GoogleGenerativeAI(model="gemini-2.0-flash",api_key=api_key['key'],temperature=0.2)

## Loading PDFs (articles from our research group)

In [6]:
paths = []
paths_folder_create = []

for root, dirs, files in os.walk("Data"):
    for f in files:
        if f.endswith('.pdf'):
            paths.append(os.path.join(root,f))
            paths_folder_create.append(os.path.join(os.getcwd(),"ToDataBase",f))

We create a folder for each article to pre-process the references

In [7]:
os.makedirs(os.path.join(os.getcwd(), "ToDataBase"), exist_ok=True)
[os.makedirs(re.sub(r'.pdf', '', p), exist_ok=True) for p in paths_folder_create]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

In [ ]:
# We read the articles from our scientific group with PyPDFLoader in single mode (all content)

articles = []

for file in paths:
    loader = loader = PyPDFLoader(file, mode="single")
    doc = loader.load()[0] # As this function provides a list, we select the first element.
    doc.page_content = hf.clean_text(doc.page_content)
    
    articles.append(doc)

### Json generation

In [10]:
class paper(BaseModel):

    PaperTitle: str = Field(description="The full title of the research paper")
    Publication: str = Field(description="Year: The year the paper was published")
    Authors: str = Field(description="The full names of all authors of the paper")
    Email: str = Field(description="The email address of the author (if provided)")
    Abstract: str = Field(description="The full text of the paper's abstract.")
    Introduction: str = Field(description="The full text fo the paper's introduction. Bear in mind that it can have other names such as background.")
    Methods: str = Field(description="The full text fo the paper's methods. Don't take the information from abstract.")
    Results: str = Field(description="The full text fo the paper's results. Don't take the information from abstract. Please take all possible text of results, this section could be divided into different sections.")
    Discussion: str = Field(description="The full text fo the paper's discussion if provided. Otherwise leave the filed blank. It should be entitled 'Discussion'")
    Conclusion: str = Field(description="The full text fo the paper's conclusion if provided. Otherwise leave the filed blank. Please don't take this information form abstract.")
    DOI: str = Field(description="Provide the Digital Object Identifier (DOI) for the paper. If a DOI is not available, provide a URL for the paper. This field is compulsory. Example of a DOI format: 10.1007/s10814-017-9105-3, 10.1016/j.cell.2023.08.001, 10.1038/nature12345.")
    Journal: str = Field(description="provide the name of the journal, e.g Nature.")

In [11]:
parser = JsonOutputParser(pydantic_object=paper)

prompt = PromptTemplate(
    template="""
You are an expert in analyzing scientific research papers. Please carefully read the provided research paper above and extract the following key information:
Extract these nine (10) properties from the research paper:

Paper Title: The full title of the research paper

Publication Year: The year the paper was published

Authors: The full names of all authors of the paper

Email: The email address of the author (if provided)

Abstract: The full text of the paper's abstract

Introduction: The full text fo the paper's introduction. Bear in mind that it can have other names such as background.

Methods: The full text fo the paper's methods. Don't take the information from abstract.

Results: The full text fo the paper's results. Don't take the information from abstract. Please take all possible text of results, this section could be divided into different sections.

Discussion:The full text fo the paper's discussion if provided. Otherwise leave the filed blank. It shoudl be entitled "Discussion"

Conclusion: The full text fo the paper's conclusion if provided. Otherwise leave the filed blank. Please don't take this information form abstract.

DOI: Provide the Digital Object Identifier (DOI) for the paper. If a DOI is not available, provide a URL for the paper. This field is compulsory. Example of a DOI format: 10.1007/s10814-017-9105-3, 10.1016/j.cell.2023.08.001, 10.1038/nature12345.

Journal: provide the name of the journal, e.g Nature.


Guidelines:


The extracted information should be factual and accurate to the document. Be extremely concise, except for the Abstract, Introduction, Methods, Results. Discussion and Conclusion which should be copied in full.
The extracted entities should be self-contained and easily understood without the rest of the paper. If a property is missing from the paper, please leave the field empty rather than guessing.
Answer in JSON format. The JSON should contain 9 keys: "PaperTitle", "PublicationYear", "Authors", "Email", "Abstract", "Introduction", "Methods","Results", "DOI", "Reference". You MUST add the DOI since is key to identify the article.

Format instructions: \n{format_instructions}\n
    
The article is this:\n{query}\n""",

    input_variables=["query"],

    partial_variables={"format_instructions": parser.get_format_instructions()}
)

To ensure all DOI are retrieved with perform a doble check.

In [67]:
chain = prompt | llm | parser

doi_prompt = PromptTemplate(
    template=(
        """Extract the Digital Object Identifier (DOI) from the following paper.
        "The DOI must be returned in its raw form (e.g., 10.1007/s10814-017-9105-3,
        "10.1016/j.cell.2023.08.001, 10.1038/nature12345).
        "Return only the DOI string with no additional text.
        "If no DOI is found, return None.\n\n
        "Paper:\n{query}"""
    ),
    input_variables=["query"]
)

chain_doi = doi_prompt | llm

In [ ]:
my_json = []

for doc in tqdm.tqdm(articles, total=len(articles)):
    info_article = chain.invoke({"query": doc.page_content})

    if 'DOI' not in info_article.keys():
        previous_doi= chain_doi.invoke({"query":doc.page_content})
        if 'https://doi.org/' not in previous_doi:
            info_article['DOI'] = 'https://doi.org/' + previous_doi
        else:
            info_article['DOI'] = previous_doi
    
    my_json.append(info_article)

with open('info_articles_main.pkl','wb') as f:
    pkl.dump(my_json,f)

100%|██████████| 18/18 [11:53<00:00, 39.62s/it]


## Getting references of each article

In [ ]:
# Already DOI (To avoid repeatin previously included articles)

includedDOI = [ re.sub('(https://doi\.org/|http://dx\.doi\.org/)', '',a['DOI']) for a in my_json]

In [ ]:
my_json_ref = []
pmid_dict = {}
not_available = []

HEADERS = {
        "User-Agent": (
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
            "AppleWebKit/537.36 (KHTML, like Gecko) "
            "Chrome/115.0.0.0 Safari/537.36"
        )
    }

for i, article in enumerate(my_json):
    
    path_name = re.sub(r'.pdf','',paths[i]).split('/')[1]
    pmid_dict[path_name] = []

    print(f"We process the article: {path_name}")

    doi = hf.clean_doi(article['DOI'])
    
    cross_ref = requests.get('https://api.crossref.org/works/'+doi)
    list_reference = cross_ref.json()['message'].get('reference',None)
    article['List_references'] = list(set([x.get('DOI') for x in list_reference if x.get('DOI') and x.get('DOI') not in includedDOI]))

    # We retrieve the pmid from each DOI

    for ref_doi in article['List_references']:
            
        pmid_dict[path_name].append(hf.get_pmid_from_doi(ref_doi))

    # We try to get the pdf

    print("Getting PDFs-----------------------------\n")

    output_dir = os.path.join("ToDataBase", path_name)

    if len(pmid_dict[path_name]) > 0:

        command = [
        "python3", "-m", "pubmed2pdf", "pdf",
        "--out", output_dir,
        "--pmids", f"{', '.join(pmid_dict[path_name])}",
        "--maxtries","1",
        "--errors", os.path.join(output_dir,"pubmed2pdf_log.txt")
        ]
        subprocess.run(command)

        # We process pdfs

        pdfs2retrieve = [os.path.join(output_dir,f) for f in os.listdir(output_dir) if f.endswith('.pdf')]

        print("Turn PDFs into JSON--------------------\n")

        for pdf in tqdm.tqdm(pdfs2retrieve, total=len(pdfs2retrieve)):

            loader = loader = PyPDFLoader(pdf, mode="single")
            doc = loader.load()[0] # As this function provides a list, we select the first element.
            doc.page_content = hf.clean_text(doc.page_content)

            try:
                info_article = chain.invoke({"query": doc.page_content})
                doi_candidate = info_article.get("DOI",None)

                if not doi_candidate:

                    retry_doi = chain_doi.invoke({"query":doc.page_content})
                    info_article['DOI'] = 'https://doi.org/' + retry_doi if 'https://doi.org/' not in retry_doi else retry_doi

                else:
                    info_article['DOI'] = 'https://doi.org/' + info_article['DOI'] if 'https://doi.org/' not in info_article['DOI'] else info_article['DOI']

                my_json_ref.append(info_article)
            except Exception:
                not_available.append(re.sub(r".pdf","",os.path.basename(pdf)))

        # We try to get the html otherwise

        print("Getting xml from PMC--------------------\n")

        log_file = os.path.join(output_dir,"pubmed2pdf_log.txt")
        if os.path.exists(log_file):
            to_retrieve = pd.read_csv(log_file, header=None,dtype='str')[0].tolist() # We get those files that could not be retrieved
        else:
            to_retrieve = []

        to_retrieve.extend([re.sub(r'.html','',f)  for f in os.listdir(output_dir) if f.endswith('.html')]) # We add those in html format since they do not have information


        # Clean information
        for f in os.listdir(output_dir):
            if f.endswith('html'):
                os.remove(os.path.join(output_dir,f)) # Then, we remove from directory
        
        print("Turn XML into JSON--------------------\n")
        
        for pmid in tqdm.tqdm(to_retrieve, total=len(to_retrieve)):

            try:
                pmcid = hf.get_pmcid_from_pmid(pmid)
                response = requests.get(url = f"https://www.ncbi.nlm.nih.gov/pmc/articles/PMC{pmcid}/", headers=HEADERS)
                soup = BeautifulSoup(response.text, "html.parser")
                text_paper = soup.find("article").get_text(separator="\n", strip=True)
                info_article = chain.invoke({"query": text_paper})

                doi_candidate = info_article.get("DOI",None)
                if not doi_candidate:
                    retry_doi= chain_doi.invoke({"query": text_paper})
                    info_article['DOI'] = 'https://doi.org/' + retry_doi if 'https://doi.org/' not in retry_doi else retry_doi
                        
                else:
                    info_article['DOI'] = 'https://doi.org/' + info_article['DOI'] if 'https://doi.org/' not in info_article['DOI'] else info_article['DOI']

                my_json_ref.append(info_article)
                
            except Exception:
                not_available.append(pmid)

            time.sleep(1)
    

    includedDOI.extend(article['List_references'])

    # Save the results

    with open("info_articles_ref_final.pkl","wb") as f:
        pkl.dump(my_json_ref, f)

We process the article: Devesa-Peiro2020
Getting PDFs-----------------------------

We process the article: Sebastian-Leon2025
Getting PDFs-----------------------------

We process the article: Henarejos-Castillo2020
Getting PDFs-----------------------------

We process the article: Parraga-Leo2023
Getting PDFs-----------------------------

We process the article: Garcia-Acero2025
Getting PDFs-----------------------------

We process the article: Sebastian-Leon2021
Getting PDFs-----------------------------

We process the article: Henarejos-Castillo2021
Getting PDFs-----------------------------



2025-08-17 21:24:52,774 - INFO - pubmed2pdf.cli - Trying to fetch pmid 21989058
2025-08-17 21:24:53,518 - INFO - pubmed2pdf.cli - Trying to fetch pmid 27901055
2025-08-17 21:24:53,518 - INFO - pubmed2pdf.cli - Trying to fetch pmid 24020646
2025-08-17 21:24:53,518 - INFO - pubmed2pdf.cli - Trying to fetch pmid 25227694
2025-08-17 21:24:55,788 - INFO - pubmed2pdf.cli - Trying to fetch pmid 11125122
2025-08-17 21:24:56,503 - INFO - pubmed2pdf.cli - Trying to fetch pmid None
2025-08-17 21:24:57,337 - INFO - pubmed2pdf.cli - Trying to fetch pmid 26243799
2025-08-17 21:24:58,065 - INFO - pubmed2pdf.cli - Trying to fetch pmid 30276597
2025-08-17 21:25:00,376 - INFO - pubmed2pdf.cli - Trying to fetch pmid 32461654
2025-08-17 21:25:00,376 - INFO - pubmed2pdf.cli - Trying to fetch pmid 26620551
2025-08-17 21:25:01,311 - INFO - pubmed2pdf.cli - Trying to fetch pmid None
2025-08-17 21:25:02,131 - INFO - pubmed2pdf.cli - Trying to fetch pmid 16722528
2025-08-17 21:25:02,131 - INFO - pubmed2pdf.cli 

Done downloading. All downloaded can be found in ToDataBase/Henarejos-Castillo2021
Turn PDFs into JSON--------------------



100%|██████████| 20/20 [09:05<00:00, 27.29s/it]


Getting xml from PMC--------------------

Turn XML into JSON--------------------



...

2025-08-18 00:25:18,509 - INFO - pubmed2pdf.cli - Trying to fetch pmid 27128483
2025-08-18 00:25:21,327 - INFO - pubmed2pdf.cli - Trying to fetch pmid 11120680
2025-08-18 00:25:22,005 - INFO - pubmed2pdf.cli - Trying to fetch pmid None
2025-08-18 00:25:22,764 - INFO - pubmed2pdf.cli - Trying to fetch pmid None
2025-08-18 00:25:24,516 - INFO - pubmed2pdf.cli - Trying to fetch pmid 16005454
2025-08-18 00:25:25,886 - INFO - pubmed2pdf.cli - Trying to fetch pmid 18252602
2025-08-18 00:25:27,146 - INFO - pubmed2pdf.cli - Trying to fetch pmid None
2025-08-18 00:25:27,907 - INFO - pubmed2pdf.cli - Trying to fetch pmid 26385059
2025-08-18 00:25:29,064 - INFO - pubmed2pdf.cli - Trying to fetch pmid 20689021
2025-08-18 00:25:29,812 - INFO - pubmed2pdf.cli - Trying to fetch pmid 24706003
2025-08-18 00:25:30,940 - INFO - pubmed2pdf.cli - Trying to fetch pmid 27122490


Done downloading. All downloaded can be found in ToDataBase/Diaz-Gimeno2017
Turn PDFs into JSON--------------------



100%|██████████| 1/1 [00:41<00:00, 41.76s/it]


Getting xml from PMC--------------------

Turn XML into JSON--------------------



100%|██████████| 10/10 [01:43<00:00, 10.39s/it]
